In [52]:
import os, logging, time
import pandas as pd
import scrapy
import requests
import numpy as np
import boto3
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import plotly.express as px

## Scrape hotels informations by cities

In [ ]:
cities = [
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle",
]

In [5]:
os.chdir("scrap_booking")
os.getcwd()

'/home/triomphant/Documents/KPS Consulting/jedha/collection/Kayak/scrap_booking'

In [159]:
!scrapy crawl Booking_info -O results/residences.json

2024-06-17 03:25:19 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrap_booking)
2024-06-17 03:25:19 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.7 (main, Dec 15 2023, 18:12:31) [GCC 11.2.0], pyOpenSSL 24.0.0 (OpenSSL 3.0.13 30 Jan 2024), cryptography 42.0.2, Platform Linux-6.2.0-39-generic-x86_64-with-glibc2.37
2024-06-17 03:25:19 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'BOT_NAME': 'scrap_booking',
 'DOWNLOAD_DELAY': 3,
 'FEED_EXPORT_ENCODING': 'utf-8',
 'HTTPCACHE_ENABLED': True,
 'NEWSPIDER_MODULE': 'scrap_booking.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'RETRY_TIMES': 6,
 'SPIDER_MODULES': ['scrap_booking.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2024-06-17 03:25:19 [asyncio] DEBUG: Using selector: EpollSelector
2024-06-17 03:25:19 [scrapy.utils.log] DEBUG: Using reactor: twisted.inter

In [6]:
df_residences = pd.read_json('results/residences.json')

In [7]:
df_residences.head()

,city,info
0,Mont+Saint+Michel,"[{'name': 'La Vieille Auberge', 'description':..."
1,St+Malo,"[{'name': 'Hôtel Ar Iniz', 'description': 'L'é..."
2,Bayeux,[{'name': 'Gites les Pourquoi Pas - Résidence ...
3,Le+Havre,"[{'name': 'Loft Design, proximité plage', 'des..."
4,Amiens,[{'name': 'L'hortillon de Saint Leu / Studio C...


In [8]:
# Explode the 'Info' column into multiple rows
df_residences = df_residences.explode("info")

# Create a new DataFrame with the rearranged information
df_residences_new = pd.DataFrame(df_residences["info"].tolist())

# Create a new DataFrame with the rearranged information
df_residences_new["city"] = df_residences["city"].to_list()

# Rearrange the columns as needed
df_residences = df_residences_new[
    ["city", "name", "url", "score", "picture"]
]

In [163]:
df_residences['url'].to_csv("residence_urls.csv")

In [47]:
len(df_residences["url"])

875

In [41]:
!scrapy crawl Booking_residence_detail -O results/residences_details.json

2024-06-19 17:30:22 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrap_booking)
2024-06-19 17:30:22 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.7 (main, Dec 15 2023, 18:12:31) [GCC 11.2.0], pyOpenSSL 24.0.0 (OpenSSL 3.0.13 30 Jan 2024), cryptography 42.0.2, Platform Linux-6.2.0-39-generic-x86_64-with-glibc2.37
2024-06-19 17:30:22 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'BOT_NAME': 'scrap_booking',
 'DOWNLOAD_DELAY': 1,
 'FEED_EXPORT_ENCODING': 'utf-8',
 'HTTPCACHE_ENABLED': True,
 'NEWSPIDER_MODULE': 'scrap_booking.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'RETRY_TIMES': 6,
 'SPIDER_MODULES': ['scrap_booking.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2024-06-19 17:30:22 [asyncio] DEBUG: Using selector: EpollSelector
2024-06-19 17:30:22 [scrapy.utils.log] DEBUG: Using reactor: twisted.inter

In [42]:
# Load initial URLs
df_residences_details_raw = pd.read_json("results/residences_details.json")

# Clean up the location information
df_residences_details_raw["location"] = df_residences_details_raw["location"].apply(
    lambda x: x.replace("\n", "") if isinstance(x, str) else x
)

# Initialize the DataFrame for correct results
df_residences_details_correct = df_residences_details_raw[
    df_residences_details_raw["name"].notnull()
].reset_index(drop=True)

# Initialize incorrect URLs
incorrect_urls = df_residences_details_raw[df_residences_details_raw["name"].isnull()]["url"]
incorrect_urls.to_csv('residences_incorrect_urls.csv')
incorrect_urls = incorrect_urls.to_list()

In [49]:
df_residences.merge(
    df_residences_details_correct[
        ["url", "name", "location", "strengths", "longitude", "latitude"]
    ],
    how="left",
    left_on="name",
    right_on="name",
).to_csv("residence_full_infos.csv")

In [62]:
# Clean up the residences informations and save them
df_full_residences_data = pd.read_csv("residence_full_infos.csv", index_col=0)
df_clean_residences_data = df_full_residences_data[(df_full_residences_data["location"]).notnull()].reset_index(drop=True)


df_clean_residences_data.to_csv("residences_clean_data.csv")

In [63]:
df_clean_residences_data.index

RangeIndex(start=0, stop=868, step=1)

## Get coordinates for each city

In [79]:
city_list = []
failed_cities = []

for city in cities:
    r = requests.get(
        f"https://nominatim.openstreetmap.org/search?city={city}&format=json"
    )
    if r.status_code == 200:
        city_info = {
            "city" : city,
            "city_latitude" : r.json()[0]["lat"],
            "city_longitude" : r.json()[0]["lon"]        
        }
        city_list.append(city_info)
    else:
        failed_cities.append(city)

In [ ]:
df_city_coord = pd.DataFrame(city_list)
df_city_coord.to_csv("city_coordinates.csv")

## Get weather infos for each city

In [76]:
df_city_coord = pd.read_csv("city_coordinates.csv", index_col=0)

In [30]:
from dotenv import dotenv_values
from datetime import datetime
config = dotenv_values(".env")
OW_API_KEY = config['OW_API_KEY']
AWS_API_KEY = config['AWS_ACCESS_KEY']
AWS_SECRET_KEY = config['AWS_SECRET_ACCESS_KEY']
USERNAME = config['USERNAME']
PASSWORD = config['PASSWORD']
HOSTNAME = config["HOSTNAME"]
DBNAME = config["DBNAME"]

In [82]:
# Fetch the 5-day weather forecast for a given city using the OpenWeatherMap API
def fetch_weather_data(city):
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={city['city_latitude']}&lon={city['city_longitude']}&exclude=minutely,hourly,alerts&appid={OW_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {"city": city['city'], "data": data}

weather_data = df_city_coord.apply(fetch_weather_data, axis=1)
weather_data.to_csv("weather_data.csv")

In [83]:
# Parse the weather forecast data into a DataFrame
def parse_weather_data(data):
    weather_list = data["daily"]
    rows = []
    for entry in weather_list:
        row = {
            "date": datetime.fromtimestamp(entry["dt"]).date(),
            "temp": entry["temp"]["day"],
            "humidity": entry["humidity"],
            "rain": entry["rain"] if "rain" in entry else 0,
            "pop": entry["pop"],
            "clouds": entry["clouds"],
        }
        rows.append(row)
    return pd.DataFrame(rows)

weather_dfs = {
    city_data["city"]: parse_weather_data(city_data["data"])
    for city_data in weather_data
}
# Add the city name to the df
for city, weather_df in weather_dfs.items():
    weather_df["city"] = city


##### The units of the variables have differents units then let us normalize them

In [84]:
weather_dfs = {
    city_data["city"]: parse_weather_data(city_data['data']) for city_data in weather_data
}

for city, weather_df in weather_dfs.items():
    weather_df['city'] = city

#### Let's compute the score 
We assume that a nice weather is when it is sunny and not raining


#### To compute score we compute 4 * temperature - 100*(1-* probability of rain)

In [85]:
# Calculate a weather score for each entry in the DataFrame
def score_weather(df):
    df["score"] = df.apply(
        lambda row: (4* row["temp"])
        + (1 - row["pop"])*100,
        axis=1,
    )
    return df


weather_dfs = {city: score_weather(df) for city, df in weather_dfs.items()}

# Concatenate weather data for all cities

df_city_weather = pd.concat(weather_dfs.values())

In [86]:

df_city_coord['city_id'] = df_city_coord.index
# Add city index to daily scores
df_final_weather = df_city_weather.merge(
    df_city_coord, how="outer", left_on="city", right_on="city"
)

In [87]:
# Compute the number days from the current day
df_final_weather["day_number"] = (
    pd.to_datetime(df_final_weather["date"]) - pd.Timestamp(datetime.now().date())
).dt.days

In [88]:
# Rename the columns
df_final_weather.rename(
    columns={
        "temp": "city_temp",
        "humidity": "city_humidity",
        "rain": "city_rain",
        "pop": "city_pop",
        "clouds": "city_clouds",
    }, inplace= True)

In [89]:
df_final_weather = df_final_weather[["city_id", "city", "city_latitude", "city_longitude","date", "day_number", "city_temp", "city_humidity", "city_rain", "city_pop", "city_clouds", "score"]]

In [91]:
df_final_weather.to_csv("clean_results/weather.csv")

In [64]:
df_clean_residences_data.to_csv("clean_results/residences.csv")

In [8]:
# Load weather data and clean residences data
df_final_weather = pd.read_csv("clean_results/weather.csv", index_col=0)
df_clean_residences_data = pd.read_csv("clean_results/residences.csv", index_col=0)

#### Store residences and Weather data into S3

In [ ]:
session = boto3.Session(aws_access_key_id= AWS_API_KEY, aws_secret_access_key=AWS_SECRET_KEY)
s3 = session.resource('s3')
bucket = s3.Bucket("kayakproject")

In [17]:
bucket.put_object(Key="Residences.csv", Body=df_clean_residences_data.to_csv())

s3.Object(bucket_name='mykayakproject', key='Residences.csv')

In [18]:
bucket.put_object(Key="Weather.csv", Body=df_final_weather.to_csv())

s3.Object(bucket_name='mykayakproject', key='Weather.csv')

### ETL

Load data into a RDS database

In [21]:
residence_final = pd.read_csv("s3://kayakproject/Residences.csv", index_col=0)

In [26]:
residence_final.iloc[:2]

,city,name,url_x,score,picture,url_y,location,strengths,longitude,latitude
0,Mont+Saint+Michel,La Vieille Auberge,https://www.booking.com/hotel/fr/la-vieille-au...,"7,4",https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/la-vieille-au...,"Grande Rue, 50170 Le Mont-Saint-Michel, France","['Parking', 'Connexion Wi-Fi gratuite', 'Resta...",-1.511457,48.636063
1,Mont+Saint+Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,"8,1",https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/le-relais-du-...,"La Caserne, 50170 Le Mont-Saint-Michel, France","['Parking privé', 'Chambres familiales', 'Conn...",-1.510906,48.616263


In [ ]:
engine = create_engine(
    f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True
)

In [35]:
# Storing the two dataframes as SQL tables
df_final_weather.to_sql("weather", con=engine, if_exists="replace", index=False)
df_clean_residences_data.to_sql("hotels", con=engine, if_exists="replace", index=False)

2025-01-05 23:09:43,425 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-01-05 23:09:43,426 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-05 23:09:43,439 INFO sqlalchemy.engine.Engine select current_schema()
2025-01-05 23:09:43,440 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-05 23:09:43,454 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-01-05 23:09:43,455 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-05 23:09:43,468 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-05 23:09:43,479 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

868

Load data from database

In [ ]:
query = text("SELECT * FROM weather LIMIT 15;")
pd.read_sql(query, engine)

2025-01-05 23:12:11,457 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-05 23:12:11,459 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-01-05 23:12:11,459 INFO sqlalchemy.engine.Engine [cached since 148s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x79e627b48e00>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-01-05 23:12:11,460 INFO sqlalchemy.engine.Engine SELECT * FROM weather LIMIT 15;
2025-01-05 23:12:11,461 INFO sqlalchemy.engine.Engine [generated in 0.00109s] {}
2025-01-05 23

,city_id,city,city_latitude,city_longitude,date,day_number,city_temp,city_humidity,city_rain,city_pop,city_clouds,score
0,0,Mont Saint Michel,48.635954,-1.511460,2024-06-19,0,17.23,82,3.76,1.00,100,68.92
1,0,Mont Saint Michel,48.635954,-1.511460,2024-06-20,1,21.17,75,7.00,1.00,57,84.68
2,0,Mont Saint Michel,48.635954,-1.511460,2024-06-21,2,18.81,72,0.24,0.74,20,101.24
3,0,Mont Saint Michel,48.635954,-1.511460,2024-06-22,3,16.02,73,5.65,1.00,78,64.08
4,0,Mont Saint Michel,48.635954,-1.511460,2024-06-23,4,19.36,73,0.10,0.20,86,157.44
5,0,Mont Saint Michel,48.635954,-1.511460,2024-06-24,5,22.20,68,0.00,0.00,13,188.80
6,0,Mont Saint Michel,48.635954,-1.511460,2024-06-25,6,24.70,62,0.00,0.00,0,198.80
7,0,Mont Saint Michel,48.635954,-1.511460,2024-06-26,7,26.51,64,0.21,0.30,12,176.04
8,1,St Malo,49.314695,-96.953823,2024-06-19,0,18.52,53,2.19,1.00,1,74.08
9,1,St Malo,49.314695,-96.953823,2024-06-20,1,20.90,61,0.00,0.00,4,183.60


In [41]:
query = text("SELECT * FROM hotels LIMIT 5;")
pd.read_sql(query, engine)

2025-01-05 23:13:04,947 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-05 23:13:04,949 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-01-05 23:13:04,950 INFO sqlalchemy.engine.Engine [cached since 201.5s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x79e627b4aea0>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-01-05 23:13:04,951 INFO sqlalchemy.engine.Engine SELECT * FROM hotels LIMIT 5;
2025-01-05 23:13:04,952 INFO sqlalchemy.engine.Engine [generated in 0.00090s] {}
2025-01-05 23

,city,name,url_x,score,picture,url_y,location,strengths,longitude,latitude
0,Mont+Saint+Michel,La Vieille Auberge,https://www.booking.com/hotel/fr/la-vieille-au...,"7,4",https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/la-vieille-au...,"Grande Rue, 50170 Le Mont-Saint-Michel, France","['Parking', 'Connexion Wi-Fi gratuite', 'Resta...",-1.511457,48.636063
1,Mont+Saint+Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,"8,1",https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/le-relais-du-...,"La Caserne, 50170 Le Mont-Saint-Michel, France","['Parking privé', 'Chambres familiales', 'Conn...",-1.510906,48.616263
2,Mont+Saint+Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"8,2",https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/mont-saint-mi...,"La Caserne, 50170 Le Mont-Saint-Michel, France","['Parking privé', 'Chambres familiales', 'Rest...",-1.510545,48.614247
3,Mont+Saint+Michel,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,"7,1",https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/la-mere-poula...,"Grande Rue, 50170 Le Mont-Saint-Michel, France","['Parking', 'Chambres familiales', 'Connexion ...",-1.510540,48.635085
4,Mont+Saint+Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,"7,4",https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/les-terrasses...,"Grande Rue, 50170 Le Mont-Saint-Michel, France","['Parking', 'Connexion Wi-Fi gratuite', 'Resta...",-1.510379,48.635349


#### Visualization

In [48]:
df_final_weather.head(15)

,city_id,city,city_latitude,city_longitude,date,day_number,city_temp,city_humidity,city_rain,city_pop,city_clouds,score
0,0,Mont Saint Michel,48.635954,-1.511460,2024-06-19,0,17.23,82,3.76,1.00,100,68.92
1,0,Mont Saint Michel,48.635954,-1.511460,2024-06-20,1,21.17,75,7.00,1.00,57,84.68
2,0,Mont Saint Michel,48.635954,-1.511460,2024-06-21,2,18.81,72,0.24,0.74,20,101.24
3,0,Mont Saint Michel,48.635954,-1.511460,2024-06-22,3,16.02,73,5.65,1.00,78,64.08
4,0,Mont Saint Michel,48.635954,-1.511460,2024-06-23,4,19.36,73,0.10,0.20,86,157.44
5,0,Mont Saint Michel,48.635954,-1.511460,2024-06-24,5,22.20,68,0.00,0.00,13,188.80
6,0,Mont Saint Michel,48.635954,-1.511460,2024-06-25,6,24.70,62,0.00,0.00,0,198.80
7,0,Mont Saint Michel,48.635954,-1.511460,2024-06-26,7,26.51,64,0.21,0.30,12,176.04
8,1,St Malo,49.314695,-96.953823,2024-06-19,0,18.52,53,2.19,1.00,1,74.08
9,1,St Malo,49.314695,-96.953823,2024-06-20,1,20.90,61,0.00,0.00,4,183.60


In [ ]:
top_cities = (
    df_final_weather
    .groupby("day_number")
    .apply(lambda x: x.nlargest(5, "score"))
    .reset_index(drop=True)
)

/tmp/ipykernel_19498/534925152.py:4: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [115]:
px.set_mapbox_access_token(open(".mapbox_token").read())

# Créer la carte interactive avec animation
fig = px.scatter_mapbox(
    top_cities,
    lat="city_latitude",
    lon="city_longitude",
    color="score",
    size="score",
    color_continuous_scale=px.colors.sequential.Bluered,
    range_color=[
        min(top_cities["score"]),
        max(top_cities["score"]),
    ],
    hover_name="city",
    hover_data={
        "city_temp": True,
        "city_humidity": True,
        "city_rain": True,
        "city_pop": True,
        "city_clouds": True,
        "score": True,
        "city_latitude": False,
        "city_longitude": False,
        "city": False,
        "day_number": False,
    },
    animation_frame="day_number",
    zoom=4.8
)

fig.update_layout(
    width=1100,
    height=800,
    title_x=0.5,
    template="plotly_dark",
    title_text="The 5 cities with the best weather in the next 7 days",
)
fig.show()

/home/triomphant/.local/lib/python3.12/site-packages/plotly/express/_core.py:1992: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [116]:
# Take the top city over the 7 days
top_cities_name = top_cities["city"].unique()

In [118]:
top_cities_name_text = [city.replace(" ", "+") for city in top_cities_name]

In [ ]:
# Convert score into a float
df_clean_residences_data["score"] = df_clean_residences_data["score"].str.replace(",", ".").astype(float)
# Replace null and nan score by -1
df_clean_residences_data["score"].fillna(-1, inplace=True)
df_clean_residences_data["score"] = df_clean_residences_data["score"].astype(int)

best_hotels_best_weather = (
    df_clean_residences_data[df_clean_residences_data["city"].isin(top_cities_name_text)]
    .groupby("city")
    .apply(lambda x: x.nlargest(20, "score"))
    .reset_index(drop=True)
)

/tmp/ipykernel_19498/898629025.py:10: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [132]:
best_hotels_best_weather.head()

,city,name,url_x,score,picture,url_y,location,strengths,longitude,latitude
0,Aigues+Mortes,Le 8 - Maison - Jardin - BBQ - Aigues Mortes,https://www.booking.com/hotel/fr/le-8-maison-j...,10.0,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/le-8-maison-j...,"8 Rue du Fbg de la Fraternité, 30220 Aigues-Mo...","['Parking gratuit', 'Connexion Wi-Fi gratuite'...",4.187893,43.569317
1,Aigues+Mortes,Evasion,https://www.booking.com/hotel/fr/evasion-aigue...,10.0,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/evasion-aigue...,"356 Rue Jacques CÅur, 30220 Aigues-Mortes, Fr...","['Piscine extérieure', 'Parking gratuit', 'Con...",4.182881,43.575810
2,Aigues+Mortes,Au Cœur des Remparts,https://www.booking.com/hotel/fr/au-coeur-des-...,9.9,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/au-coeur-des-...,"15 rue Roger Salengro, 30220 Aigues-Mortes, Fr...","['Parking gratuit', 'Chambres non-fumeurs']",4.192973,43.565401
3,Aigues+Mortes,Lou Mirèio,https://www.booking.com/hotel/fr/42-rue-pasteu...,9.5,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/42-rue-pasteu...,"42 Rue Pasteur, 30220 Aigues-Mortes, France","['Parking gratuit', 'Connexion Wi-Fi gratuite']",4.192579,43.566075
4,Aigues+Mortes,Mas de Constance 3pieces standing,https://www.booking.com/hotel/fr/mas-de-consta...,9.4,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/fr/mas-de-consta...,"346 Rue du Vidourle, 30220 Aigues-Mortes, France","['Piscine extérieure', 'Parking gratuit', 'Con...",4.186526,43.572115


In [145]:
# Créer la carte interactive avec animation
fig = px.scatter_mapbox(
    best_hotels_best_weather,
    lat="latitude",
    lon="longitude",
    size_max=15,
    color="score",
    color_continuous_scale=px.colors.sequential.deep,
    range_color=[
        min(best_hotels_best_weather["score"]),
        max(best_hotels_best_weather["score"]),
    ],
    hover_name="name",
    hover_data={
        "city": True,
        "strengths": True,
        "score": True,
        "url_x": False,
        "url_y": False,
        "city": False,
    },
    zoom=4.8,
)

fig.update_layout(
    width=1100,
    height=800,
    title_x=0.5,
    template="plotly_dark",
    title_text="The 5 cities with the best weather in the next 7 days",
)
fig.show()